In [68]:
import pandas as pd

ratings = pd.read_csv('/home/supay/datos/rating.csv')
ratings = ratings[['user_id','anime_id','rating']]
movies = pd.read_csv('/home/supay/datos/anime.csv')
movies = movies[['anime_id','name']]

#ratings = ratings.sample(n = 10000)
ratings = ratings.head(15000)
ratings = pd.merge(movies, ratings)

#dif = ratings.anime_id.unique()
#print(len(dif))

In [69]:
userRatings = ratings.pivot_table(index=['user_id'],columns=['name'],values='rating')

In [70]:
corrMatrix = userRatings.corr(method='pearson', min_periods=5)
corrMatrix

name,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,.hack//Gift,.hack//Intermezzo,.hack//Liminality,.hack//Quantum,...,ef: A Tale of Melodies. - Prologue,ef: A Tale of Memories.,ef: A Tale of Memories. - Prologue,gdgd Fairies,gdgd Fairies 2,s.CRY.ed,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Rou,xxxHOLiC Shunmuki
name,,,,,,,,,,,,,,,,,,,,,
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
&quot;Bungaku Shoujo&quot; Memoire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
&quot;Bungaku Shoujo&quot; Movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//G.U. Returner,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
.hack//G.U. Trilogy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
s.CRY.ed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xxxHOLiC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
xxxHOLiC Kei,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
myRatings = userRatings.loc[1].dropna()
myRatings.head(10)

name
11eyes                     -1.0
11eyes: Momoiro Genmutan   -1.0
Absolute Duo               -1.0
Accel World                -1.0
Akame ga Kill!             -1.0
Akaneiro ni Somaru Saka    -1.0
Akuma no Riddle            -1.0
Amagi Brilliant Park       -1.0
Angel Beats!               -1.0
Angel Beats! Specials      -1.0
Name: 1, dtype: float64

In [72]:
simCandidates = pd.Series()
print(simCandidates)
for i in range(0, len(myRatings.index)):
    print ("Añadiendo pelis similares a " + myRatings.index[i] + "...")
    # Recuperar las pelis similares a las calificadas
    sims = corrMatrix[myRatings.index[i]].dropna()
    # Escalar la similaridad multiplicando por la calificación de la persona
    sims = sims.map(lambda x: x * myRatings[i])
    # Añadir el puntaje a la lista de candidatos similares
    simCandidates = simCandidates.append(sims)
    
#Mirar los resultados:
#print ("ordenando...")
simCandidates.sort_values(inplace = True, ascending = False)
print (simCandidates.head(10))

<ipython-input-72-4a448cecd77e>:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  simCandidates = pd.Series()


Series([], dtype: float64)
Añadiendo pelis similares a 11eyes...
Añadiendo pelis similares a 11eyes: Momoiro Genmutan...
Añadiendo pelis similares a Absolute Duo...
Añadiendo pelis similares a Accel World...
Añadiendo pelis similares a Akame ga Kill!...
Añadiendo pelis similares a Akaneiro ni Somaru Saka...
Añadiendo pelis similares a Akuma no Riddle...
Añadiendo pelis similares a Amagi Brilliant Park...
Añadiendo pelis similares a Angel Beats!...
Añadiendo pelis similares a Angel Beats! Specials...
Añadiendo pelis similares a Angel Beats!: Another Epilogue...
Añadiendo pelis similares a Ao no Exorcist...
Añadiendo pelis similares a Ao no Exorcist Movie...
Añadiendo pelis similares a Ao no Exorcist: Kuro no Iede...
Añadiendo pelis similares a Asu no Yoichi!...
Añadiendo pelis similares a Black Bullet...
Añadiendo pelis similares a Blade &amp; Soul...
Añadiendo pelis similares a Boku wa Tomodachi ga Sukunai...
Añadiendo pelis similares a Boku wa Tomodachi ga Sukunai Episode 0...
Añadien

Añadiendo pelis similares a _Summer...
Okusama ga Seitokaichou!    10.000000
High School DxD             10.000000
Highschool of the Dead      10.000000
High School DxD New         10.000000
Ixion Saga DT               10.000000
Sword Art Online            10.000000
White Album 2                9.981116
Mayo Chiki!                  9.979317
Strange+                     9.975298
Astarotte no Omocha!         9.973233
dtype: float64


In [73]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()

In [74]:
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(10)

Maken-Ki!                          33.185788
Kimi no Iru Machi                  31.203771
Gantz                              29.503981
Jormungand: Perfect Order          28.363083
Shijou Saikyou no Deshi Kenichi    26.795942
Akatsuki no Yona                   25.698573
Dog Days                           25.051323
Dog Days&#039;                     24.447550
Nourin                             24.004273
Jormungand                         23.954880
dtype: float64

In [75]:
#print(myRatings)
filteredSims = simCandidates.drop(myRatings.index,errors='ignore')
filteredSims.head(10)

Maken-Ki!                          33.185788
Kimi no Iru Machi                  31.203771
Gantz                              29.503981
Jormungand: Perfect Order          28.363083
Shijou Saikyou no Deshi Kenichi    26.795942
Akatsuki no Yona                   25.698573
Dog Days                           25.051323
Dog Days&#039;                     24.447550
Nourin                             24.004273
Jormungand                         23.954880
dtype: float64